In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing

In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
train_df[train_df["target"]==0]["text"].values[1]

'I love fruits'

In [4]:
train_df["text"].values[1]

'Forest fire near La Ronge Sask. Canada'

In [5]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [6]:
import tensorflow_hub as hub


In [7]:
!pip install tensorflow-text

     |████████████████████████████████| 4.9 MB 5.2 MB/s 


In [8]:
import tensorflow_text as text


In [9]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)

In [10]:
bert_encoder = hub.KerasLayer(encoder_url)


In [11]:
train_df.groupby('target').describe()

id                            ...                         
         count         mean          std  ...     50%     75%      max
target                                    ...                         
0       4342.0  5276.446338  3157.206802  ...  5243.5  8038.5  10848.0
1       3271.0  5661.608071  3097.094809  ...  5676.0  8252.0  10873.0

[2 rows x 8 columns]

In [12]:
def get_sentence_embeding(sentences):
    preprocessed_text = bert_preprocess_model(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [13]:
e = get_sentence_embeding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [14]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9911089]], dtype=float32)

In [15]:
import tensorflow as tf

In [16]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [18]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df["text"],train_df["target"], stratify=train_df["target"])

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
179/179 [==============================] - 2515s 14s/step - loss: 0.6348 - accuracy: 0.6395
Epoch 2/10
179/179 [==============================] - 2471s 14s/step - loss: 0.5999 - accuracy: 0.6928
Epoch 3/10
179/179 [==============================] - 2460s 14s/step - loss: 0.5830 - accuracy: 0.7035
Epoch 4/10
179/179 [==============================] - 2456s 14s/step - loss: 0.5713 - accuracy: 0.7108
Epoch 5/10
179/179 [==============================] - 2458s 14s/step - loss: 0.5633 - accuracy: 0.7220
Epoch 6/10
179/179 [==============================] - 2464s 14s/step - loss: 0.5526 - accuracy: 0.7341
Epoch 7/10
179/179 [==============================] - 2465s 14s/step - loss: 0.5506 - accuracy: 0.7306
Epoch 8/10
179/179 [==============================] - 2466s 14s/step - loss: 0.5463 - accuracy: 0.7336
Epoch 9/10
179/179 [==============================] - 2468s 14s/step - loss: 0.5413 - accuracy: 0.7423
Epoch 10/10
115/179 [==================>...........] - ETA: 14:44 - loss:

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")

In [ ]:
sample_submission["target"] = model.predict(test_df["text"])

In [ ]:
model.save("twitter.h5")